In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from utils import *

/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

In [4]:
huggingface_models = {"base": "bert-base-uncased",
                        "large": "bert-large-uncased",
                      "base-ft-sst2": "yoshitomo-matsubara/bert-base-uncased-sst2",
                        "large-ft-sst2": "yoshitomo-matsubara/bert-large-uncased-sst2",
                      "base-ft-stsb": "gchhablani/bert-base-cased-finetuned-stsb",
                        "large-ft-stsb": "yoshitomo-matsubara/bert-large-uncased-stsb",
                      "base-ft-mrpc": "textattack/bert-base-uncased-MRPC",
                        "large-ft-mrpc": "yoshitomo-matsubara/bert-large-uncased-mrpc",
                      "base-ft-cola": "yoshitomo-matsubara/bert-base-uncased-cola",
                        "large-ft-cola": "yoshitomo-matsubara/bert-large-uncased-cola",
                      "base-ft-qnli": "gchhablani/bert-base-cased-finetuned-qnli",
                        "large-ft-qnli": "yoshitomo-matsubara/bert-large-uncased-qnli",
                      "base-ft-mnli": "yoshitomo-matsubara/bert-base-uncased-mnli",
                        "large-ft-mnli": "yoshitomo-matsubara/bert-large-uncased-mnli",
                      "base-ft-rte": "anirudh21/bert-base-uncased-finetuned-rte",
                        "large-ft-rte": "yoshitomo-matsubara/bert-large-uncased-rte",
                      "base-ft-qqp": "A-bhimany-u08/bert-base-cased-qqp",
                        "large-ft-qqp": "yoshitomo-matsubara/bert-large-uncased-qqp",
                      "base-ft-wnli": "gchhablani/bert-base-cased-finetuned-wnli",
                        "large-ft-wnli": "yoshitomo-matsubara/bert-large-uncased-wnli",
                      }

In [5]:
model_base, tokenizer = get_classification_bert_model(pre_trained_model_name=huggingface_models["base-ft-mrpc"])
train_dataloader, validation_dataloader, dataset = get_glue_task_dataset('mrpc', tokenizer)
model_LC = get_LC_model_bert(model_base, num_GP_layers=4)
model_LC.load_state_dict(torch.load("/scr/models/LC/models_archive/Bert/base-ft-mrpc/4_mrpc.pth"))
model_base.load_state_dict(torch.load("/scr/models/LC/models_archive/Bert/base-ft-mrpc/orig_mrpc.pth"))
collapse_model(model_LC)

Setting up gated layer bert.encoder.layer.11
Setting up gated layer bert.encoder.layer.10
Setting up gated layer bert.encoder.layer.9
Setting up gated layer bert.encoder.layer.8
Collapsing layer bert.encoder.layer.11
Collapsing layer bert.encoder.layer.10
Collapsing layer bert.encoder.layer.9
Collapsing layer bert.encoder.layer.8


In [6]:
print("base model parameters: ", get_num_parameters(model_base))

base model parameters:  109483778


In [7]:
standard_evaluate(model_base, dataset["validation"], "mrpc")

/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.7286306619644165,
 'eval_accuracy': 0.8308823529411765,
 'eval_f1': 0.8848080133555927,
 'eval_combined_score': 0.8578451831483846,
 'eval_runtime': 6.689,
 'eval_samples_per_second': 60.995,
 'eval_steps_per_second': 2.541}

In [7]:
def prune_l1_model(model, amount):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
    return model

In [8]:
# remove
def remove_prune(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            prune.remove(module, name='weight')
    return model 

In [9]:
def finetune(model, epochs, task, lr=0.0001):
    train_with_trainer(model, dataset["train"], dataset["validation"], epochs=epochs,
                    eval_batch_size=64, train_batch_size=64, gp_weight=0, learning_rate=lr, task=task, lr_decay=0, weight_decay=0, use_sgd=True)

In [15]:
pruned_base = prune_l1_model(model_base, amount=0.5)

RuntimeError: indices should be either on cpu or on the same device as the indexed tensor (cpu)

In [14]:
standard_evaluate(pruned_base, dataset["validation"], "mrpc")

{'eval_loss': 0.47482338547706604,
 'eval_accuracy': 0.8333333333333334,
 'eval_f1': 0.8870431893687708,
 'eval_combined_score': 0.8601882613510521,
 'eval_runtime': 1.7877,
 'eval_samples_per_second': 228.221,
 'eval_steps_per_second': 9.509}

In [20]:
finetune(pruned_base, 1, "mrpc")

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
  File "/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/transformers/models/bert/modeling_bert.py", line 1564, in forward
    outputs = self.bert(
  File "/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/soheil/miniconda3/envs/GP/lib/python3.10/si

In [19]:
pruned_base = remove_prune(pruned_base)

ValueError: Parameter 'weight' of module Linear(in_features=768, out_features=768, bias=True) has to be pruned before pruning can be removed

In [19]:
print("base model parameters: ", get_num_parameters(pruned_base, count_nonzero_only=True))

base model parameters:  tensor(66720770, device='cuda:0')


In [20]:
standard_evaluate(pruned_base, dataset["validation"], "mrpc") 

/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.4852569103240967,
 'eval_accuracy': 0.8014705882352942,
 'eval_f1': 0.8661157024793389,
 'eval_combined_score': 0.8337931453573165,
 'eval_runtime': 3.0481,
 'eval_samples_per_second': 133.853,
 'eval_steps_per_second': 5.577}

In [21]:
pruned_LC = prune_l1_model(model_LC, amount=0.5)

In [22]:
standard_evaluate(pruned_LC, dataset["validation"], "mrpc")

/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.7729309797286987,
 'eval_accuracy': 0.7573529411764706,
 'eval_f1': 0.8455538221528862,
 'eval_combined_score': 0.8014533816646784,
 'eval_runtime': 2.8252,
 'eval_samples_per_second': 144.416,
 'eval_steps_per_second': 6.017}

In [23]:
pruned_LC = remove_prune(pruned_LC)

In [24]:
print("LC model parameters: ", get_num_parameters(pruned_LC, count_nonzero_only=True))

LC model parameters:  tensor(58450946, device='cuda:0')


In [25]:
standard_evaluate(pruned_LC, dataset["validation"], "mrpc")

/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.7729309797286987,
 'eval_accuracy': 0.7573529411764706,
 'eval_f1': 0.8455538221528862,
 'eval_combined_score': 0.8014533816646784,
 'eval_runtime': 2.7899,
 'eval_samples_per_second': 146.24,
 'eval_steps_per_second': 6.093}

In [ ]:
train_with_trainer(model, dataset["train"], dataset["validation"], epochs=0.2,
                    eval_batch_size=64, train_batch_size=64, gp_weight=0, learning_rate=0.0001, task="mrpc", lr_decay=0, weight_decay=0, use_sgd=True)